Análisis y procesamiento de señales - Trabajo semanal Nº 8
Aramayo Yohana Valentina


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig

# =========================================================
# Funciones (las mismas que ya usás)
# =========================================================

def blackman_tukey(x, M=None):
    x = x.ravel()
    N = len(x)

    if M is None:
        M = N // 10

    r_len = 2 * M - 1
    xx = x[:r_len]

    r = np.correlate(xx, xx, mode='same') / r_len
    Px = np.abs(np.fft.fft(r * sig.windows.blackman(r_len), n=N))

    return Px


def procesar_señal(señal, fs, titulo="PSD de la señal"):
    señal = señal / np.std(señal)
    N = len(señal)

    ff = np.linspace(0, fs, N, endpoint=False)
    psd = blackman_tukey(señal)

    psd_half = psd[:N // 2]
    ff_half = ff[:N // 2]

    energia_total = np.sum(psd_half)
    energia_acumulada = np.cumsum(psd_half) / energia_total

    f_95 = ff_half[np.where(energia_acumulada >= 0.95)[0][0]]
    f_98 = ff_half[np.where(energia_acumulada >= 0.98)[0][0]]

    # Gráfico
    plt.figure()
    plt.plot(ff_half, 10 * np.log10(psd_half + 1e-10))
    plt.axvline(f_95, color='r', linestyle='--', label=f'95%: {f_95:.2f} Hz')
    plt.axvline(f_98, color='g', linestyle='--', label=f'98%: {f_98:.2f} Hz')
    plt.title(titulo)
    plt.xlabel("Frecuencia [Hz]")
    plt.ylabel("PSD [dB]")
    plt.grid()
    plt.legend()
    plt.tight_layout()
    plt.show()

    return psd, ff, f_95, f_98, energia_total, energia_acumulada


# =========================================================
# Lectura EEG OpenBCI
# =========================================================

datos = pd.read_csv(
    r"C:\Users\Vale\Documents\APS\APS_vale\TPS\s01_ex01_s01.txt",
    sep=",",
    skiprows=5
)

eeg = datos.iloc[:, 1:5]
eeg = eeg.apply(pd.to_numeric).values

fs = 200  # Hz

# Quitar offset
eeg = eeg - np.mean(eeg, axis=0)

# Tiempo
n = eeg.shape[0]
tiempo = np.arange(n) / fs

# Ventana de 2 segundos
muestras = int(2 * fs)
eeg = eeg[:muestras]
tiempo = tiempo[:muestras]


ModuleNotFoundError: No module named 'pandas'